In [ ]:
#Resize  working window
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Importing libraries needed to manipulate and visualizing data 
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts, dim
import panel as pn
import panel.widgets as pnw
pn.extension()
hv.extension('bokeh')

df0 = pd.read_csv('fadn_at_a_glance.csv')

In [ ]:
tran_SYS03 = {'< 15': 0, '15 - <40': 1, '40 - <100': 2, '100 - <200': 3, '200 - <500': 4, '500 - <1000': 5,
              '1000 - <2000': 6, '2000 - <3000': 7, '3000 - <4000': 8}
rvs = {v: k for k, v in tran_SYS03.items()}

In [ ]:
df0['pop_cat'] = df0['SYS03'].replace(tran_SYS03)

In [ ]:
df1 = df0.groupby(['TF14', 'COUNTRY'])['pop_cat'].mean().reset_index()

In [ ]:
df1['pop_cat'] = df1['pop_cat'].round()

In [ ]:
df1['hols'] =  df1['pop_cat'].replace(rvs)

In [ ]:
ds = hv.Dataset(df1, kdims=['TF14', 'COUNTRY'], vdims=['pop_cat','hols']).sort()

In [ ]:
heatmap = hv.HeatMap(ds)
heatmap.opts(opts.HeatMap(cmap='coolwarm',height=500, width=500,xrotation=45,tools=['hover'],title='Click on cell for info'))
posxy = hv.streams.Tap(source=heatmap, x='(16) Specialist other fieldcrops', y='(ELL) Greece')

def tap_curves(x,y):
    df2 = df0.query('COUNTRY == "'+ y +'" and TF14 =="' + x +'"')
    ds1 = hv.Dataset(df2, kdims=['TF14', 'COUNTRY'])
    cur420 =  hv.Curve(ds1,'YEAR', 'SE420', label='SE420')
    cur131 =  hv.Curve(ds1,'YEAR', 'SE131', label='SE131')
    cur605 =  hv.Curve(ds1,'YEAR', 'SE605', label='SE605')
    cur270 =  hv.Curve(ds1,'YEAR', 'SE270', label='SE270')
    curves = (cur420 * cur131 * cur605 * cur270).relabel(y + ' -- ' + x).opts(height=300,width=500, ylabel='')
    cur425 =  hv.Curve(ds1,'YEAR', 'SE425',label='SE425').opts(height=200,width=500)
    cur011 =  hv.Curve(ds1,'YEAR', 'SE011',label='SE011').opts(height=200,width=500)
    cur132 =  hv.Curve(ds1,'YEAR', 'SE132',label='SE132').opts(height=200,width=500)
#     layout = (curves + cur425 + cur011 + cur132).cols(1)
#     layout.opts(opts.Curve(framewise=True),
#                 opts.Overlay(legend_position='top_left'))
    return cur420

# def tap_areas(x,y):
#     df2 = df0.query('COUNTRY == "'+ y +'" and TF14 =="' + x +'"')
#     ds1 = hv.Dataset(df2, kdims=['TF14', 'COUNTRY'])
#     over1 = hv.Area(ds1, 'YEAR','plus').relabel('SE(131+600+405)') * hv.Area(df2, 'YEAR','miners').opts(fill_alpha=0.5).relabel('SE(275+360+365)')
#     cur1 = hv.Curve(ds1,'YEAR', 'SE420')
#     over2 =  over1 * cur1
#     ar281 = hv.Area(ds1, 'YEAR','SE281',label='SE281')
#     ar336 = hv.Area(ds1, 'YEAR','SE336',label='SE336')
#     ar360 = hv.Area(ds1, 'YEAR','SE360',label='SE360')
#     ar370 = hv.Area(ds1, 'YEAR','SE370',label='SE370')
#     ar375 = hv.Area(ds1, 'YEAR','SE375',label='SE375')
#     ar380 = hv.Area(ds1, 'YEAR','SE380',label='SE380')
#     over3 = hv.Area.stack(ar281 * ar336 * ar360 * ar370 * ar375 * ar380)

#     ar135 = hv.Area(ds1, 'YEAR','SE135',label='SE135')
#     ar206 = hv.Area(ds1, 'YEAR','SE206',label='SE206')
#     ar256 = hv.Area(ds1, 'YEAR','SE256',label='SE256')
#     ar600 = hv.Area(ds1, 'YEAR','SE600',label='SE600')
#     ar405 = hv.Area(ds1, 'YEAR','SE405',label='SE405')
#     over4 =  hv.Area.stack(ar135 * ar206 * ar256 * ar600 * ar405)
#     layout = (over2 + over3 + over4).cols(1)
#     layout.opts(opts.Curve(line_dash='dotted', color='g'),
#                 opts.Area(framewise=True, ylabel='', xrotation=45, width=500),
#                opts.Overlay(legend_position='top_left'))
#     return layout

# dmap1 = hv.DynamicMap(tap_curves, streams=[posxy])
# dmap2 = hv.DynamicMap(tap_areas, streams=[posxy])

pn.Row(hv.DynamicMap(tap_curves, streams=[posxy]),heatmap).servable()